In [17]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import random
from torchvision.io import read_image
from torchvision import transforms
from torchvision import models
import os

In [18]:
lr = 1e-3
batch_size = 64

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

trans = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.491, 0.482, 0.447), (0.247, 0.243, 0.262))
])

root = '' ## 이미지 경로

original_trainset = datasets.ImageFolder(root=root+"/origin-cifar-10/train", transform=trans)
original_testset = datasets.ImageFolder(root=root+"origin-cifar-10/test", transform=trans)
svd_trainset = datasets.ImageFolder(root=root+"/svd-cifar-10/train", transform=trans)
svd_testset = datasets.ImageFolder(root=root+"/svd-cifar-10/test", transform=trans)
svd4_trainset = datasets.ImageFolder(root=root+"/svd4-cifar-10/train", transform=trans)
svd4_testset = datasets.ImageFolder(root=root+"/svd4-cifar-10/test", transform=trans)

o_trainloader = DataLoader(original_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
o_testloader = DataLoader(original_testset, batch_size=batch_size, shuffle=False, drop_last=False)
s_trainloader = DataLoader(svd_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
s_testloader = DataLoader(svd_testset, batch_size=batch_size, shuffle=False, drop_last=False)
s4_trainloader = DataLoader(svd4_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
s4_testloader = DataLoader(svd4_testset, batch_size=batch_size, shuffle=False, drop_last=False)

classes = original_testset.classes

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model1 = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
model2 = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
model3 = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)

model1.classifier = nn.Linear(in_features=model1.classifier.in_features, out_features=10, bias=True)
model2.classifier = nn.Linear(in_features=model2.classifier.in_features, out_features=10, bias=True)
model3.classifier = nn.Linear(in_features=model3.classifier.in_features, out_features=10, bias=True)

nn.init.xavier_normal_(model1.classifier.weight)
nn.init.xavier_normal_(model2.classifier.weight)
nn.init.xavier_normal_(model3.classifier.weight)

model1.to(device)
model2.to(device)
model3.to(device)

Using cuda device


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=lr)
optimizer2 = optim.Adam(model2.parameters(), lr=lr)
optimizer3 = optim.Adam(model3.parameters(), lr=lr)

In [20]:
def train_loop(dataloader, model, criterion, optimizer):
    size = len(dataloader.dataset)
    loss_sum = 0
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X.to(device))
        loss = criterion(pred, y.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            loss_sum += loss
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

    return loss_sum / size

def test_loop(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    acc, loss = 0, 0
    conf_matrix = []
    for i in range(10):
        conf_matrix.append([0, 0, 0, 0])

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(device))
            loss += criterion(pred, y.to(device)).item()
            acc += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()
            temp = pred.argmax(1)
            for k in range(len(temp)):
                for i in range(10):
                    if i == temp[k]:
                        if i == y[k]:
                            conf_matrix[i][0] += 1.0
                        else:
                            conf_matrix[i][1] += 1.0
                    else:
                        if i == y[k]:
                            conf_matrix[i][2] += 1.0
                        else:
                            conf_matrix[i][3] += 1.0

    
    loss /= (size/batch_size)
    acc /= size
    print(f"Test Error: \n Accuracy: {(100*acc):>0.1f}%, Avg loss: {loss:>8f}\n")

    return conf_matrix

In [21]:
epochs = 20
train_loss1, test_res1, test_res2, test_res3 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model1.train()
    train_loss1.append(train_loop(o_trainloader, model1, criterion, optimizer1))
    model1.eval()
    test_res1.append(test_loop(o_testloader, model1, criterion))
    test_res2.append(test_loop(s_testloader, model1, criterion))
    test_res3.append(test_loop(s4_testloader, model1, criterion))

Epoch 1-------------------------------
loss: 3.280354 [    0/50000]
loss: 1.236742 [ 6400/50000]
loss: 0.913350 [12800/50000]
loss: 0.812873 [19200/50000]
loss: 0.691420 [25600/50000]
loss: 1.054644 [32000/50000]
loss: 0.963739 [38400/50000]
loss: 0.720646 [44800/50000]
Test Error: 
 Accuracy: 73.7%, Avg loss: 0.789634

Test Error: 
 Accuracy: 68.6%, Avg loss: 0.929127

Test Error: 
 Accuracy: 51.1%, Avg loss: 1.498784

Epoch 2-------------------------------
loss: 0.778071 [    0/50000]
loss: 1.131401 [ 6400/50000]
loss: 0.716596 [12800/50000]
loss: 0.520589 [19200/50000]
loss: 0.535386 [25600/50000]
loss: 0.840052 [32000/50000]
loss: 0.692780 [38400/50000]
loss: 1.036360 [44800/50000]
Test Error: 
 Accuracy: 75.5%, Avg loss: 0.728699

Test Error: 
 Accuracy: 70.8%, Avg loss: 0.865127

Test Error: 
 Accuracy: 54.5%, Avg loss: 1.437181

Epoch 3-------------------------------
loss: 0.662257 [    0/50000]
loss: 0.410095 [ 6400/50000]
loss: 0.595747 [12800/50000]
loss: 0.464279 [19200/5000

In [22]:
epochs = 20
train_loss2, test_res4, test_res5, test_res6 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model2.train()
    train_loss2.append(train_loop(s_trainloader, model2, criterion, optimizer2))
    model2.eval()
    test_res4.append(test_loop(o_testloader, model2, criterion))
    test_res5.append(test_loop(s_testloader, model2, criterion))
    test_res6.append(test_loop(s4_testloader, model2, criterion))

Epoch 1-------------------------------
loss: 2.974874 [    0/50000]
loss: 1.516981 [ 6400/50000]
loss: 1.028115 [12800/50000]
loss: 1.072624 [19200/50000]
loss: 0.979327 [25600/50000]
loss: 1.029349 [32000/50000]
loss: 0.646198 [38400/50000]
loss: 0.856517 [44800/50000]
Test Error: 
 Accuracy: 71.2%, Avg loss: 0.837637

Test Error: 
 Accuracy: 72.3%, Avg loss: 0.806953

Test Error: 
 Accuracy: 64.4%, Avg loss: 1.020402

Epoch 2-------------------------------
loss: 0.702213 [    0/50000]
loss: 0.667175 [ 6400/50000]
loss: 0.713269 [12800/50000]
loss: 0.544219 [19200/50000]
loss: 0.933740 [25600/50000]
loss: 1.108415 [32000/50000]
loss: 1.832025 [38400/50000]
loss: 0.933980 [44800/50000]
Test Error: 
 Accuracy: 69.2%, Avg loss: 0.908094

Test Error: 
 Accuracy: 67.0%, Avg loss: 0.952874

Test Error: 
 Accuracy: 55.1%, Avg loss: 1.304982

Epoch 3-------------------------------
loss: 1.016918 [    0/50000]
loss: 0.821108 [ 6400/50000]
loss: 0.673520 [12800/50000]
loss: 0.571246 [19200/5000

In [30]:
epochs = 20
train_loss3, test_res7, test_res8, test_res9 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model3.train()
    #train_loss3.append(train_loop(s4_trainloader, model3, criterion, optimizer3))
    model3.eval()
    test_res7.append(test_loop(o_testloader, model3, criterion))
    test_res8.append(test_loop(s_testloader, model3, criterion))
    test_res9.append(test_loop(s4_testloader, model3, criterion))

Epoch 1-------------------------------
Test Error: 
 Accuracy: 73.5%, Avg loss: 3.422359

Test Error: 
 Accuracy: 74.5%, Avg loss: 2.999991

Test Error: 
 Accuracy: 73.8%, Avg loss: 1.866480

Epoch 2-------------------------------
Test Error: 
 Accuracy: 73.5%, Avg loss: 3.422359

Test Error: 
 Accuracy: 74.5%, Avg loss: 2.999991

Test Error: 
 Accuracy: 73.8%, Avg loss: 1.866480

Epoch 3-------------------------------
Test Error: 
 Accuracy: 73.5%, Avg loss: 3.422359

Test Error: 
 Accuracy: 74.5%, Avg loss: 2.999991

Test Error: 
 Accuracy: 73.8%, Avg loss: 1.866480

Epoch 4-------------------------------
Test Error: 
 Accuracy: 73.5%, Avg loss: 3.422359

Test Error: 
 Accuracy: 74.5%, Avg loss: 2.999991

Test Error: 
 Accuracy: 73.8%, Avg loss: 1.866480

Epoch 5-------------------------------
Test Error: 
 Accuracy: 73.5%, Avg loss: 3.422359

Test Error: 
 Accuracy: 74.5%, Avg loss: 2.999991

Test Error: 
 Accuracy: 73.8%, Avg loss: 1.866480

Epoch 6-------------------------------
T

In [24]:
print("Result for Testing with original datasets")
print(max(test_res1), max(test_res2), max(test_res3))

print("Result for Testing with original datasets")
print(max(test_res4), max(test_res5), max(test_res6))

print("Result for Testing with original datasets")
print(max(test_res7), max(test_res8), max(test_res9))

Result for Testing with original datasets
[[904.0, 306.0, 96.0, 8694.0], [911.0, 105.0, 89.0, 8895.0], [779.0, 263.0, 221.0, 8737.0], [667.0, 358.0, 333.0, 8642.0], [850.0, 252.0, 150.0, 8748.0], [695.0, 176.0, 305.0, 8824.0], [878.0, 139.0, 122.0, 8861.0], [820.0, 75.0, 180.0, 8925.0], [791.0, 65.0, 209.0, 8935.0], [863.0, 103.0, 137.0, 8897.0]] [[884.0, 269.0, 116.0, 8731.0], [851.0, 88.0, 149.0, 8912.0], [725.0, 277.0, 275.0, 8723.0], [668.0, 481.0, 332.0, 8519.0], [846.0, 387.0, 154.0, 8613.0], [682.0, 222.0, 318.0, 8778.0], [745.0, 82.0, 255.0, 8918.0], [764.0, 88.0, 236.0, 8912.0], [809.0, 100.0, 191.0, 8900.0], [868.0, 164.0, 132.0, 8836.0]] [[742.0, 262.0, 258.0, 8738.0], [577.0, 45.0, 423.0, 8955.0], [632.0, 455.0, 368.0, 8545.0], [670.0, 934.0, 330.0, 8066.0], [816.0, 858.0, 184.0, 8142.0], [478.0, 226.0, 522.0, 8774.0], [295.0, 32.0, 705.0, 8968.0], [529.0, 55.0, 471.0, 8945.0], [781.0, 299.0, 219.0, 8701.0], [849.0, 465.0, 151.0, 8535.0]]
Result for Testing with original da

In [25]:
def best_acc_finder(test_res):
    best, max = 0, 0
    for i in range(len(test_res)):
        correct = 0.0
        total = 0.0
        for j in range(len(test_res[i])):
            correct = test_res[i][j][0] +  test_res[i][j][3]
            total = correct + test_res[i][j][2] + test_res[i][j][1]
            if max < (correct / total):
                max = correct / total
                best = i

    print(f"best accuary: {max:>0.2f} at {best}")
    return best

In [26]:
best_acc_finder(test_res1)
best_acc_finder(test_res2)
best_acc_finder(test_res3)
best_acc_finder(test_res4)
best_acc_finder(test_res5)
best_acc_finder(test_res6)
best_acc_finder(test_res7)
best_acc_finder(test_res8)
best_acc_finder(test_res9)

best accuary: 0.98 at 12
best accuary: 0.98 at 7
best accuary: 0.96 at 2
best accuary: 0.98 at 16
best accuary: 0.98 at 10
best accuary: 0.96 at 18
best accuary: 0.97 at 18
best accuary: 0.97 at 18
best accuary: 0.97 at 15


15

In [27]:
def metrics_single_class(conf_mat):
    accuracy = (conf_mat[0] + conf_mat[3]) / (conf_mat[0] + conf_mat[1] + conf_mat[2] + conf_mat[3])
    precision = conf_mat[0] / (conf_mat[0] + conf_mat[1])
    recall = conf_mat[0] / (conf_mat[0] + conf_mat[2])
    f1_score = 2 * precision * recall / (precision + recall)
    
    return accuracy, precision, recall, f1_score

def metrics(conf_mats):
    sum_acc, sum_pre, sum_recall, sum_f1 = 0.0, 0.0, 0.0, 0.0
    for i in range(len(conf_mats)):
        temp_acc, temp_pre, temp_recall, temp_f1 = metrics_single_class(conf_mats[i])
        #print(f"For class-{i}:\n Accuracy: {temp_acc:>0.4f}, Precision: {temp_pre:>0.4f}, Recall: {temp_recall:>0.4f}, F1-score: {temp_f1:>0.4f}")
        sum_acc += temp_acc
        sum_pre += temp_pre
        sum_recall += temp_recall
        sum_f1 += temp_f1

    sum_acc /= 10
    sum_pre /= 10
    sum_recall /= 10
    sum_f1 /= 10

    return sum_acc, sum_pre, sum_recall, sum_f1


print(metrics(test_res1[best_acc_finder(test_res1)]))
print(metrics(test_res2[best_acc_finder(test_res2)]))
print(metrics(test_res3[best_acc_finder(test_res3)]))
print(metrics(test_res4[best_acc_finder(test_res4)]))
print(metrics(test_res5[best_acc_finder(test_res5)]))
print(metrics(test_res6[best_acc_finder(test_res6)]))
print(metrics(test_res7[best_acc_finder(test_res7)]))
print(metrics(test_res8[best_acc_finder(test_res8)]))
print(metrics(test_res9[best_acc_finder(test_res9)]))

best accuary: 0.98 at 12
(0.9642799999999999, 0.8254138619675961, 0.8213999999999999, 0.8221134069263835)
best accuary: 0.98 at 7
(0.9568399999999999, 0.7947097343596846, 0.7842000000000001, 0.7861517096251844)
best accuary: 0.96 at 2
(0.9185000000000001, 0.686238368642754, 0.5924999999999999, 0.5840582564357354)
best accuary: 0.98 at 16
(0.96322, 0.8192669882650551, 0.8160999999999999, 0.8167473970335652)
best accuary: 0.98 at 10
(0.959, 0.8002527552951137, 0.795, 0.7959183114736303)
best accuary: 0.96 at 18
(0.93916, 0.7233347779826247, 0.6958, 0.7005826388224354)
best accuary: 0.97 at 18
(0.95116, 0.7636477738634848, 0.7558, 0.7518287737332294)
best accuary: 0.97 at 18
(0.95304, 0.7694486555204408, 0.7651999999999999, 0.7629912921383515)
best accuary: 0.97 at 15
(0.9496400000000002, 0.7524474224391797, 0.7482, 0.7487016747460039)
